In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import seaborn as sns


In [2]:
features = pd.read_csv("features.csv")
brand = pd.read_csv("Brand_Mapping.csv")

In [3]:
random.seed = 1234
time = 4*6
df = features[['Date','Temperature','Fuel_Price','CPI','Unemployment','IsHoliday']][:time].copy()

df['Date'] = pd.date_range(start='2013-08-02', periods=len(df), freq='W-FRI')
df['Temperature'] = df['Temperature'].apply(lambda x: x+random.uniform(-10, 10))
df['Fuel_Price'] = df['Fuel_Price'].apply(lambda x: x+random.uniform(-1, 2))
df['CPI'] = df['CPI'].apply(lambda x: x+random.uniform(10,20))
df['Unemployment'] = df['Unemployment'].apply(lambda x: x+random.uniform(1, 5))

change = df.sample(2).index
df.loc[change,'IsHoliday'] = True

df['Date'] = pd.to_datetime(df['Date'])
df

,Date,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday
0,2013-08-02,42.393160,3.641399,229.413946,11.459912,False
1,2013-08-09,37.759567,3.003300,222.749271,11.744110,True
2,2013-08-16,30.206037,4.133080,228.569107,10.986181,False
3,2013-08-23,53.621159,3.329968,221.751005,9.289029,False
4,2013-08-30,47.555563,2.797669,230.936806,9.455108,False
5,2013-09-06,52.372952,3.262437,227.805463,10.020354,False
6,2013-09-13,54.882970,3.215714,228.087617,12.950632,False
7,2013-09-20,52.995069,4.144125,221.566816,10.214960,False
8,2013-09-27,59.479211,3.786770,229.389168,11.853714,False
9,2013-10-04,70.221409,2.614871,228.957745,9.077986,False


In [4]:
selected_categories = ['RTD Tea/Coffee',
                     'Energy Drinks',
                     'Sports Drinks',
                     'Dairy-based Drinks',
                     'Beverage Mixes',
                      'Dietary Supplements']

selected_brands = ['THE COCA-COLA CO.',
                 'PEPSICO INC.',
                 'HEINEKEN N.V.',
                 'SUNTORY HOLDINGS LTD.',
                 'DIAGEO PLC']

selected_regions = ['Midwest',
                   'West',
                   'South',
                   'Northeast']

In [5]:
df['Brands'] = [selected_brands] * df.shape[0]
df['Categories'] = [selected_categories] * df.shape[0]
df['Region'] = [selected_regions] * df.shape[0]
df.head(3)

,Date,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday,Brands,Categories,Region
0,2013-08-02,42.393160,3.641399,229.413946,11.459912,False,"[THE COCA-COLA CO., PEPSICO INC., HEINEKEN N.V...","[RTD Tea/Coffee, Energy Drinks, Sports Drinks,...","[Midwest, West, South, Northeast]"
1,2013-08-09,37.759567,3.003300,222.749271,11.744110,True,"[THE COCA-COLA CO., PEPSICO INC., HEINEKEN N.V...","[RTD Tea/Coffee, Energy Drinks, Sports Drinks,...","[Midwest, West, South, Northeast]"
2,2013-08-16,30.206037,4.133080,228.569107,10.986181,False,"[THE COCA-COLA CO., PEPSICO INC., HEINEKEN N.V...","[RTD Tea/Coffee, Energy Drinks, Sports Drinks,...","[Midwest, West, South, Northeast]"


In [6]:
df = df.explode('Brands')
df = df.explode('Categories')
df = df.explode('Region')
df.head(3)

,Date,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday,Brands,Categories,Region
0,2013-08-02,42.39316,3.641399,229.413946,11.459912,False,THE COCA-COLA CO.,RTD Tea/Coffee,Midwest
0,2013-08-02,42.39316,3.641399,229.413946,11.459912,False,THE COCA-COLA CO.,RTD Tea/Coffee,West
0,2013-08-02,42.39316,3.641399,229.413946,11.459912,False,THE COCA-COLA CO.,RTD Tea/Coffee,South


In [7]:
df.drop_duplicates(subset=['Date','Brands','Categories','Region'],inplace=True)
df.shape

(2880, 9)

In [8]:
# 1235,12313,1,7811
seed = 7811
def strong_trend_and_seasonality(nsample):
    ## simulate a simple sinusoidal function
#     random.seed(seed)
    x1 = np.linspace(1, 100, nsample)
    
    slope = random.uniform(.035, 0.045)
    intercept = random.uniform(1.8, 2.2)
    
    trend = intercept + slope*x1

    # Seasonal
    seasonal = np.sin(random.uniform(.35, .45)*x1)

    #noise 
    sigma = 0.2
    noise = sigma * np.random.normal(size=nsample)

    y =((trend+ seasonal + noise)*100).astype(int)
    return y

def no_pattern(lower, upper, nsample):
#     random.seed(seed)
    return np.random.randint(lower,upper,nsample)

def correlated_discounts(nsample, discount):
    ## simulate a simple sinusoidal function
#     random.seed(seed)
    x1 = np.linspace(1, 100, nsample)
    
    slope = random.uniform(.035, 0.045)
    intercept = random.uniform(1.8, 2.2)
    
    trend = intercept + slope*x1

    # Seasonal
    seasonal = np.sin(random.uniform(.35, .45)*x1)

    #noise 
    sigma = 0.2
    noise = sigma * np.random.normal(size=nsample)

    y =((trend+ seasonal + 4*discount+noise)*100).astype(int)
    return y

def lumpy_demand(df): 
#     random.seed(seed)
    col_name = "Weekly Sales"
    nan_percent = {col_name:0.025}
    
    for i, row_value in df[col_name].iteritems():
        if random.random() <= nan_percent[col_name]:
            df[col_name][i] = np.nan
    df[col_name] = df[col_name].fillna(0)
    return df[col_name]

def lagged_correlation(nsample,fuel,cpi,unemployment):
    ## simulate a simple sinusoidal function
#     random.seed(seed)
    x1 = np.linspace(1, 100, nsample)
    
    slope = random.uniform(.035, 0.045)
    intercept = random.uniform(1.8, 2.2)
    
    trend = intercept + slope*x1

    # Seasonal
    seasonal = np.sin(random.uniform(.35, .45)*x1)

    #noise 
    sigma = 0.2
    noise = sigma * np.random.normal(size=nsample)
    
    y =((trend+ seasonal + noise+fuel/fuel.sum()+cpi/cpi.sum()+unemployment/unemployment.sum())*100)

    return y
    
    
    



In [9]:
sales_df = pd.DataFrame()

total_iter = len(selected_categories)
count = 0

for cat in selected_categories:
            
    count = count + 1

    print(f"\rRunning Iteration {count}/{total_iter}...",end='')
    
#   strong trend and seasonality
    if(cat == 'RTD Tea/Coffee'):
        
        sub = df[df['Categories'] == cat].copy()            
        sub = sub.sort_values(by=['Date'],ascending=True)
        sub['Discounts'] = np.random.uniform(0.05, 0.40,sub.shape[0])
        
        sub["Weekly Sales"] = strong_trend_and_seasonality(sub.shape[0]) 
        
        replacement_mapping_dict={'THE COCA-COLA CO.':'COCOA_1',
                          'PEPSICO INC.':'COCOA_2',
                          'HEINEKEN N.V.':'COCOA_3',
                          'SUNTORY HOLDINGS LTD.':'COCOA_4',
                          'DIAGEO PLC':'COCOA_5'}

        sub["Brands"].replace(replacement_mapping_dict, inplace=True)

        
        if sales_df.shape[0]:
            sales_df = pd.concat([sales_df,sub],axis=0,ignore_index=True,sort=False)
        else:
            sales_df = sub
#   Intermittent Demand
    elif(cat == 'Energy Drinks'):
        
        sub = df[df['Categories'] == cat].copy()            
        sub = sub.sort_values(by=['Date'],ascending=True)
        sub['Discounts'] = np.random.uniform(0.05, 0.40,sub.shape[0])

        sub["Weekly Sales"] = no_pattern(random.randint(250,270),random.randint(300,310),sub.shape[0])
        
        sub["Weekly Sales"] = lumpy_demand(sub)
        
        replacement_mapping_dict={'THE COCA-COLA CO.':'FIZZY_1',
                          'PEPSICO INC.':'FIZZY_2',
                          'HEINEKEN N.V.':'FIZZY_3',
                          'SUNTORY HOLDINGS LTD.':'FIZZY_4',
                          'DIAGEO PLC':'FIZZY_5'}

        sub["Brands"].replace(replacement_mapping_dict, inplace=True)

        
        if sales_df.shape[0]:
            sales_df = pd.concat([sales_df,sub],axis=0,ignore_index=True,sort=False)
        else:
            sales_df = sub

#   Discounts correlated
    elif(cat == 'Sports Drinks'):
        
        sub = df[df['Categories'] == cat].copy()            
        sub = sub.sort_values(by=['Date'],ascending=True)
        
        sub['Discounts'] = np.random.uniform(0.05, 0.40,sub.shape[0])
        sub["Weekly Sales"] = correlated_discounts(sub.shape[0],sub['Discounts'])
        
        replacement_mapping_dict={'THE COCA-COLA CO.':'COLA_1',
                          'PEPSICO INC.':'COLA_2',
                          'HEINEKEN N.V.':'COLA_3',
                          'SUNTORY HOLDINGS LTD.':'COLA_4',
                          'DIAGEO PLC':'COLA_5'}

        sub["Brands"].replace(replacement_mapping_dict, inplace=True)        
        
        if sales_df.shape[0]:
            sales_df = pd.concat([sales_df,sub],axis=0,ignore_index=True,sort=False)
        else:
            sales_df = sub
            
#   New category 
#   MILK_5 is the new product here
    elif(cat == 'Dairy-based Drinks'):
        
        sub = df[df['Categories'] == cat].copy()            
        sub = sub.sort_values(by=['Date'],ascending=True)
        
        sub['Discounts'] = np.random.uniform(0.05, 0.40,sub.shape[0])
        
        sub["Weekly Sales"] = strong_trend_and_seasonality(sub.shape[0])
        
#         sub['Weekly Sales'] = sub.apply(lambda x: np.nan if x['Brands'] == 'DIAGEO PLC' else x['Weekly Sales'], axis = 1)
        
        replacement_mapping_dict={'THE COCA-COLA CO.':'MILK_1',
                          'PEPSICO INC.':'MILK_2',
                          'HEINEKEN N.V.':'MILK_3',
                          'SUNTORY HOLDINGS LTD.':'MILK_4',
                          'DIAGEO PLC':'MILK_5'}
        
        sub["Brands"].replace(replacement_mapping_dict, inplace=True) 
        

        if sales_df.shape[0]:
            sales_df = pd.concat([sales_df,sub],axis=0,ignore_index=True,sort=False)
        else:
            sales_df = sub
            
#   Lumpy demand        
    elif(cat == 'Beverage Mixes'):
        
        sub = df[df['Categories'] == cat].copy()            
        sub = sub.sort_values(by=['Date'],ascending=True)
        
        sub['Discounts'] = np.random.uniform(0.05, 0.40,sub.shape[0])
        
        sub["Weekly Sales"] = no_pattern(random.randint(20,60),random.randint(300,310),sub.shape[0])
        
        sub["Weekly Sales"] = lumpy_demand(sub)
        
        replacement_mapping_dict={'THE COCA-COLA CO.':'JUICES_1',
                          'PEPSICO INC.':'JUICES_2',
                          'HEINEKEN N.V.':'JUICES_3',
                          'SUNTORY HOLDINGS LTD.':'JUICES_4',
                          'DIAGEO PLC':'JUICES_5'}
        
        sub["Brands"].replace(replacement_mapping_dict, inplace=True)        

        
        if sales_df.shape[0]:
            sales_df = pd.concat([sales_df,sub],axis=0,ignore_index=True,sort=False)
        else:
            sales_df = sub

#  Lagged Correlation Index
    
    elif(cat == 'Dietary Supplements'):
        
        sub = df[df['Categories'] == cat].copy()            
        sub = sub.sort_values(by=['Date'],ascending=True)
        
        sub['Discounts'] = np.random.uniform(0.05, 0.40,sub.shape[0])
        
        sub['Fuel_Price'] = sub['Fuel_Price'].shift(4) 
        sub['CPI'] = sub['CPI'].shift(4)
        sub['Unemployment'] = sub['Unemployment'].shift(4)
        
        sub["Weekly Sales"] = lagged_correlation(sub.shape[0],sub['Fuel_Price'],sub['CPI'],sub['Unemployment'])
        
        replacement_mapping_dict={'THE COCA-COLA CO.':'PROTEIN_1',
                          'PEPSICO INC.':'PROTEIN_2',
                          'HEINEKEN N.V.':'PROTEIN_3',
                          'SUNTORY HOLDINGS LTD.':'PROTEIN_4',
                          'DIAGEO PLC':'PROTEIN_5'}
        
        sub["Brands"].replace(replacement_mapping_dict, inplace=True)        

        
        
        if sales_df.shape[0]:
            sales_df = pd.concat([sales_df,sub],axis=0,ignore_index=True,sort=False)
        else:
            sales_df = sub

    
    else:
        print('xyz')
            
            
                    
                              

Running Iteration 6/6...

<ipython-input-8-b764a0387970>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name][i] = np.nan
C:\Users\u351202\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [10]:
pip install numexpr

Note: you may need to restart the kernel to use updated packages.


In [11]:
sales_df

,Date,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday,Brands,Categories,Region,Discounts,Weekly Sales
0,2013-08-02,42.393160,3.641399,229.413946,11.459912,False,COCOA_1,RTD Tea/Coffee,Midwest,0.229018,238.000000
1,2013-08-02,42.393160,3.641399,229.413946,11.459912,False,COCOA_5,RTD Tea/Coffee,Northeast,0.128568,232.000000
2,2013-08-02,42.393160,3.641399,229.413946,11.459912,False,COCOA_5,RTD Tea/Coffee,South,0.335173,251.000000
3,2013-08-02,42.393160,3.641399,229.413946,11.459912,False,COCOA_5,RTD Tea/Coffee,West,0.099713,293.000000
4,2013-08-02,42.393160,3.641399,229.413946,11.459912,False,COCOA_5,RTD Tea/Coffee,Midwest,0.267614,263.000000
...,...,...,...,...,...,...,...,...,...,...,...
2875,2014-01-10,89.445691,3.688837,225.976881,9.557852,False,PROTEIN_1,Dietary Supplements,West,0.123040,552.032428
2876,2014-01-10,89.445691,3.688837,225.976881,9.557852,False,PROTEIN_1,Dietary Supplements,Midwest,0.256284,593.053775
2877,2014-01-10,89.445691,3.688837,225.976881,9.557852,False,PROTEIN_5,Dietary Supplements,South,0.274711,583.727234
2878,2014-01-10,89.445691,3.688837,225.976881,9.557852,False,PROTEIN_3,Dietary Supplements,Midwest,0.061849,609.467845


In [12]:
# Making SKU's column
sales_df['SKU'] = sales_df['Brands'] + sales_df['Region']
sales_df['SKU'] = pd.factorize(sales_df['SKU'])[0]+1
sales_df['SKU'] = 'SKU_' + sales_df['SKU'].astype(str)

# Imputing Nan Values
sales_df['Fuel_Price'] = sales_df['Fuel_Price'].fillna(sales_df['Fuel_Price'].mean())
sales_df['CPI'] = sales_df['CPI'].fillna(sales_df['CPI'].mean())
sales_df['Unemployment'] = sales_df['Unemployment'].fillna(sales_df['Unemployment'].mean())

# For new product scenario, keeping only those sku's which are representing new product data
# Keeping only SKU_62,SKU_63,SKU_64,SKU_65 for MILK_5
dropped_skus = ['SKU_61','SKU_66','SKU_67','SKU_68','SKU_69','SKU_70','SKU_71','SKU_72',
                'SKU_73','SKU_74','SKU_75','SKU_76','SKU_77','SKU_78','SKU_79','SKU_80']

 
sales_df = sales_df[~sales_df['SKU'].isin(dropped_skus)]


In [13]:
cols = ['Date','Brands', 'Categories', 'Region','SKU','Temperature', 
        'Fuel_Price', 'CPI', 'Unemployment', 'IsHoliday','Discounts' ]

In [14]:
sales_df[cols].to_csv("Supply_Chain_Test_dataset.csv",index=False)

In [15]:
sales_df[cols]

,Date,Brands,Categories,Region,SKU,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday,Discounts
0,2013-08-02,COCOA_1,RTD Tea/Coffee,Midwest,SKU_1,42.393160,3.641399,229.413946,11.459912,False,0.229018
1,2013-08-02,COCOA_5,RTD Tea/Coffee,Northeast,SKU_2,42.393160,3.641399,229.413946,11.459912,False,0.128568
2,2013-08-02,COCOA_5,RTD Tea/Coffee,South,SKU_3,42.393160,3.641399,229.413946,11.459912,False,0.335173
3,2013-08-02,COCOA_5,RTD Tea/Coffee,West,SKU_4,42.393160,3.641399,229.413946,11.459912,False,0.099713
4,2013-08-02,COCOA_5,RTD Tea/Coffee,Midwest,SKU_5,42.393160,3.641399,229.413946,11.459912,False,0.267614
...,...,...,...,...,...,...,...,...,...,...,...
2875,2014-01-10,PROTEIN_1,Dietary Supplements,West,SKU_119,89.445691,3.688837,225.976881,9.557852,False,0.123040
2876,2014-01-10,PROTEIN_1,Dietary Supplements,Midwest,SKU_101,89.445691,3.688837,225.976881,9.557852,False,0.256284
2877,2014-01-10,PROTEIN_5,Dietary Supplements,South,SKU_103,89.445691,3.688837,225.976881,9.557852,False,0.274711
2878,2014-01-10,PROTEIN_3,Dietary Supplements,Midwest,SKU_112,89.445691,3.688837,225.976881,9.557852,False,0.061849


In [16]:
sales_df[sales_df['Categories'] == 'RTD Tea/Coffee'].iloc[[0, -1]]

,Date,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday,Brands,Categories,Region,Discounts,Weekly Sales,SKU
0,2013-08-02,42.393160,3.641399,229.413946,11.459912,False,COCOA_1,RTD Tea/Coffee,Midwest,0.229018,238.0,SKU_1
479,2014-01-10,89.445691,3.688837,225.976881,9.557852,False,COCOA_5,RTD Tea/Coffee,Northeast,0.124771,575.0,SKU_2


In [17]:
sales_df.shape

(2496, 12)